# Step 1: Data Collection and Cleaning

## 1a). Collecting and cleaning the Florida Sex Offender Registry Data 

### Data Collection: 

Data downloaded offender list from official Florida Law Enforcement Website.
 
Florida Sex offender site: https://offender.fdle.state.fl.us/offender/sops/home.jsf
Public data file info: https://www.fdle.state.fl.us/SOPS/public_data_file 

### Data Cleaning: 

In [1]:
import pandas as pd

# define file path for downloaded dataset (source: https://www.fdle.state.fl.us/SOPS/public_data_file) 
# and the output file
file_path = 'FloridaSOR.csv'
output_file = 'FloridaSOR_Clean.csv'

# create dataframe from downloaded date
try:
    df = pd.read_csv(file_path)
except FileNotFoundError:
    raise FileNotFoundError(f"Error: The file {file_path} was not found.")

# define mappings for race and sex fields to standardize
race_mapping = {
    'W': 'White',
    'B': 'Black or African American',
    'I': 'American Indian or Alaska Native',
    'U': 'Unknown or Unspecified',
    'A': 'Asian'
}

sex_mapping = {
    'M': 'Male',
    'F': 'Female',
    'U': 'Unknown or Unspecified'
}

# copy the original DataFrame to a new one for cleaning
df_clean = df.copy()

# apply the mapping to race and sex columns to standardize
df_clean['RACE'] = df['RACE'].map(race_mapping)
df_clean['SEX'] = df['SEX'].map(sex_mapping)

# concatenate address lines and remove any leading/trailing whitespace
df_clean['PERM_ADDRESS'] = df[['PERM_ADDRESS_LINE_1', 'PERM_ADDRESS_LINE_2']].fillna('').agg(' '.join, axis=1).str.strip()
df_clean['TEMP_ADDRESS'] = df[['TEMP_ADDRESS_LINE_1', 'TEMP_ADDRESS_LINE_2']].fillna('').agg(' '.join, axis=1).str.strip()
df_clean['TRANS_ADDRESS'] = df[['TRANS_ADDRESS_LINE_1', 'TRANS_ADDRESS_LINE_2']].fillna('').agg(' '.join, axis=1).str.strip()

# standardize city and county names by capitalizing the first letter of each word
for col in ['PERM_CITY', 'TEMP_CITY', 'TRANS_CITY', 'PERM_COUNTY', 'TEMP_COUNTY', 'TRANS_COUNTY']:
    df_clean[col] = df[col].str.title()

# assign the state 'Florida' as the default for address-related state fields (to standardize, currently multiple versions)
df_clean[['PERM_STATE', 'TEMP_STATE', 'TRANS_STATE']] = 'Florida'

# function to format ZIP codes correctly by combining 5-digit and 4-digit parts
def combine_zip(zip5, zip4):
    if pd.isna(zip5):
        return None
    zip5_str = str(int(zip5))
    zip4_str = str(int(zip4)).zfill(4) if pd.notna(zip4) else ''
    return f"{zip5_str}-{zip4_str}" if zip4_str and zip4_str != '0000' else zip5_str

# apply ZIP code formatting for permanent, temporary, and transitional addresses
if {'PERM_ZIP', 'PERM_ZIP4'}.issubset(df.columns):
    df_clean['PERM_ZIP'] = df.apply(lambda row: combine_zip(row['PERM_ZIP'], row['PERM_ZIP4']), axis=1)
if {'TEMP_ZIP', 'TEMP_ZIP4'}.issubset(df.columns):
    df_clean['TEMP_ZIP'] = df.apply(lambda row: combine_zip(row['TEMP_ZIP'], row['TEMP_ZIP4']), axis=1)
if {'TRANS_ZIP', 'TRANS_ZIP4'}.issubset(df.columns):
    df_clean['TRANS_ZIP'] = df.apply(lambda row: combine_zip(row['TRANS_ZIP'], row['TRANS_ZIP4']), axis=1)

# rename columns for better readability
df_clean.rename(columns={
    'TRANS_ADDRESS_ADDED': 'Trans_Address_Added_Date',
    'Department_of_Corrections_NUMBER': 'Department_of_Corrections_Number'
}, inplace=True)

# drop columns that won't be used in geospatial analysis
columns_to_drop = ['SEX', 'RACE', 'PERM_STATE', 'TEMP_STATE', 'TRANS_STATE', 'PERM_ADDRESS_LINE_1', 'PERM_ADDRESS_LINE_2',
                   'TEMP_ADDRESS_LINE_1', 'TEMP_ADDRESS_LINE_2', 'TRANS_ADDRESS_LINE_1', 'TRANS_ADDRESS_LINE_2']
df_clean.drop(columns=[col for col in columns_to_drop if col in df_clean.columns], inplace=True)

# save df to output file
df_clean.to_csv(output_file, index=False)

# done with data cleaning!
print(f"Data cleaned successfully and saved to {output_file}")


Data cleaned successfully and saved to FloridaSOR_Clean.csv


## 1b). Collecting and cleaning Florida Public School Data 

### Data Collection and Cleaning: 

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# base URL for scraping the specific webpage for Florida schools
base_url = "https://web03.fldoe.org/Schools/schoolreport.asp?id="

# dictionary mapping of school district IDs to their names
districts = {
    # mapping each ID to its corresponding district name
    1: 'Alachua', 2: 'Baker', ... , 82: 'FSU Bay'
}

# dictionary to normalize the varied names of school types into a standard format
school_type_mapping = {
    'Universities': 'University',
    'Community Colleges': 'Community College',
    'High Schools': 'High School',
    'Middle Schools': 'Middle School',
    'Elementary Schools': 'Elementary School',
    'Combination Schools': 'Combination School',
    'Other Schools': 'Other School'
}

# initialize an empty list to store data for each school
data = []

# define a function to scrape data for a given school based on its ID and district name
def scrape_data(school_id, district_name):
    # fetch the webpage
    response = requests.get(base_url + str(school_id))
    # check if the request was successful
    if response.status_code != 200:
        return  # exit the function if the webpage is unreachable
    
    # parse the HTML content of the webpage
    soup = BeautifulSoup(response.content, 'html.parser')
    # extract all table rows from the parsed HTML
    rows = soup.find_all('tr')
    school_type = None
    
    # process each row to extract relevant data
    for row in rows[4:-1]:  # skip irrelevant rows
        cells = row.text.strip().split('\n')
        if len(cells) == 1:
            # update the school type based on the first cell if it's a standalone row
            school_type = school_type_mapping.get(cells[0].strip(), cells[0].strip())
        else:
            # append a dictionary for each school containing detailed information
            data.append({
                'District': district_name,
                'School Type': school_type,
                'School Name': cells[0].strip(),
                'Address': 'Unavailable' if 'Unavailable' in cells[2] else cells[2].strip(),
                'Phone Number': cells[8].strip(),
                'Public or Private': 'Public' #set all as Public because these are public schools
            })

# iterate over each district and scrape data using the predefined function
for school_id, district_name in districts.items():
    scrape_data(school_id, district_name)

# done with data collection!
# define output file
output_file = 'FloridaPublicSchools_CLEAN.csv'

# convert the list of dictionaries into a DataFrame for further manipulation and storage
df = pd.DataFrame(data)

# manually update specific entries in the DataFrame for correctness
df.update(pd.DataFrame([
    {'District': 'Hillsborough', 'School Name': 'Dorothy C York Pk-8 Magnet School (0011)', 
     'Address': '5995 Covington Garden Dr, Apollo Beach, FL 33572', 'Phone Number': '(813) 533-2400'},
    {'District': 'St. Johns', 'School Name': 'Lakeside Academy (0562)', 
     'Address': '1455 Twin Creeks Drive, St. Augustine, FL 32095', 'Phone Number': '(904) 547-4500'},
    {'District': 'FSU Bay', 'School Name': 'Fsu Panama City District Office (9001)', 
     'Address': '4750 Collegiate Drive, Panama City, FL 32405', 'Phone Number': '(850) 872-4750'}
]))

# save df to output file
df.to_csv(output_file, index=False)

# done with data cleaning!
print(f"Data successfully scraped and saved to {output_file}")


Data successfully scraped and saved to FloridaPublicSchools_CLEAN.csv


### 1c). Collecting and cleaning Florida Private School Data 

### Data Collectiong: 
Data downloaded from Florida Department of Eduation: https://www.floridaschoolchoice.org/information/privateschooldirectory/DownloadExcelFile.aspx

### Data Cleaning: 

In [4]:
import pandas as pd

# define file path for downloaded dataset (source: https://www.floridaschoolchoice.org/information/privateschooldirectory/DownloadExcelFile.aspx) 
# and the output file
file_path = 'FloridaPrivateSchools.csv'
output_file = 'FloridaPrivateSchools_CLEAN.csv'

df = pd.read_csv(file_path)

# fill missing values with empty strings to avoid processing errors in later steps
df.fillna('', inplace=True)

# create a formatted address column by concatenating address parts and removing extra spaces
df['Address'] = df[['Address 1', 'Address 2', 'City', 'State', 'Zip']].agg(lambda x: ' '.join(x).strip(), axis=1)

# drop redundant address columns now that there is a combined and formatted address
df.drop(columns=['Address 1', 'Address 2', 'City', 'State', 'Zip'], inplace=True)

# standardize capitalization for district and school name columns to ensure consistency
df[['District', 'School Name']] = df[['District', 'School Name']].apply(lambda x: x.str.title())

# add a column indicating whether the school is public or private, set all as 'Private'
df['Public or Private'] = 'Private'

# save df to output file
df.to_csv(output_file, index=False)

# done wiht cleaning!
print(f"Data successfully cleaned and saved to {output_file}")



Data successfully cleaned and saved to FloridaPrivateSchools_CLEAN.csv


### 1d). Collecting and Cleaning Florida Child Care Centers Data 

### Data Collections:
Data downloaded from Florida Department of Children and Families: https://www.myflfamilies.com/search?q=%22child+care+provider+list%22#gsc.tab=0&gsc.q=%22child%20care%20provider%20list%22&gsc.page=1

### Data Cleaning: 

In [5]:
import pandas as pd

# define file path for downloaded dataset (source: https://www.myflfamilies.com/search?q=%22child+care+provider+list%22#gsc.tab=0&gsc.q=%22child%20care%20provider%20list%22&gsc.page=1) 
# and the output file
file_path = 'FloridaChildCareCenters.csv'
output_file = 'FloridaChildCareCenters_CLEAN.csv'

df = pd.read_csv(file_path)

# rename column for consistency
df.rename(columns={'Physical Address': 'Address'}, inplace=True)

# drop columns not needed for geospatial analysis
columns_to_drop = ['Physical Address', 'Is Head Start', 'Is Public School', 'Is School Age Only',
                   'Is Urban Zoned', 'Facility Size', 'School Readiness Status', 'Gold Seal',
                   'Fee', 'Is Faith Based', 'Is VPK', 'City', 'Zip', 'State', 'Director']
df.drop(columns=[col for col in columns_to_drop if col in df.columns], inplace=True)

# save df to output file
df.to_csv(output_file, index=False)

# done with cleaning!
print(f"Data successfully cleaned and saved to {output_file}")

Data successfully cleaned and saved to FloridaChildCareCenters_CLEAN.csv


### 1e). Collecting and Cleaning Florida Public Parks Data 

### Data Collection and Cleaning: 

In [6]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# base URL for scraping park information from Florida State Parks website
base_url = "https://www.floridastateparks.org/parks-and-trails?parks%5B0%5D=amenities%3A274&page="

# storage list for collecting park data
park_data = []

# scrape park data across multiple pages
for page in range(4):
    response = requests.get(f"{base_url}{page}")
    response.raise_for_status()  # check for HTTP request errors
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # find all park entries on the current page
    for park in soup.find_all('div', class_='card__content'):
        park_name = park.find('h3', class_='card__title').get_text(strip=True)
        park_summary = park.find('div', class_='card__summary').get_text(strip=True)
        park_address = park.find('div', class_='card__address').get_text(strip=True)
        
        # append each park's details to the park_data list
        park_data.append({
            'Park Name': park_name,
            'Summary': park_summary,
            'Address': park_address
        })

# create a DataFrame from the list of park data
df = pd.DataFrame(park_data)
output_file = 'Florida_ParksWithPlaygrounds_CLEAN.csv'
df.to_csv(output_file, index=False) 

# done with data collection and cleaning! 
print(f"Data successfully scraped and saved to {output_file}")



Data successfully scraped and saved to Florida_ParksWithPlaygrounds_CLEAN.csv
